In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pprint



data = []
write = 0
sequence = []

with open("gg") as dataset :
    for line in dataset.readlines():
        line = line.replace("\n", "")
        # if write:
        #     sequence.append(line.split("\t")[0])
        values = line.split("\t")[1].split(",")
        curr = []
        for i in values:
            if i.startswith("Start Run:"):
                write = 1
            if i.startswith("Class"):
                clas = i.replace("Class :", "")
                write = 0
                if len(sequence) > 1:
                    data.append( [sequence, clas])
                    sequence = []
            if write and not i.startswith("Start Run:"):
                curr.append(i)
        if len(curr) > 0:
            sequence.append(curr)

final_seq = np.stack(data)

# model = Sequential()

# model.add(LSTM(256, input_shape=(52,)))


/Users/egemen/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:

data = []
targets = []
temp = []
for k in final_seq:
  temp = []
  for j in k[0]:
      temp.append(int(j[1]))
  targets.append(float(k[1]))
  temp = np.asarray(temp)
  temp = np.reshape(temp, (-1,3))
  data.append(temp[:45]) 
# print(temp[:45])
data = np.asarray(data)
data = np.reshape(data, (-1,1))
# print(data[0][0])



/Users/egemen/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [ ]:

# data = tf.ragged.constant(data[:])
# targets = tf.convert_to_tensor(targets[:50])
# # print(data)
# print(data)
# print(targets)
# print(f'Type of final sequence: {type(data[0])}')
# print(f'Shape of final sequence: {data.shape[0][0][0]}')

In [ ]:
train =[]
validation = []
validation_target = []
train_target = []
test = []
test_target = []
num =round(len(data)/3)
for i in range(num *2):
  train.append(data[i][0])
  validation.append(data[i+(num)][0]) 
  

  train_target.append(targets[i])
  validation_target.append(targets[i+(num)])

for i in range(num):
  test_target.append(targets[i+(2*num)])
  test.append(data[i+(2*num)][0])

# print(test)
train =tf.ragged.constant(train)
validation = tf.ragged.constant(validation)
validation_target = tf.convert_to_tensor(validation_target)
train_target = tf.convert_to_tensor(train_target)
test = tf.ragged.constant(test)
test_target = tf.convert_to_tensor(test_target)

In [ ]:
# print(test[0])
# print(test_target)

In [ ]:
#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 45
final_seq=data

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 3)))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 256)               266240    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 266,497
Trainable params: 266,497
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# # print(train)
# train = tf.convert_to_tensor(train)
# validation = tf.convert_to_tensor(validation)
# test = tf.convert_to_tensor(test)

# train_target = tf.convert_to_tensor(train_target)
# # train_target = (train_target+1)/2

# validation_target = tf.convert_to_tensor(validation_target)
# # validation_target = (validation_target+1)/2

# test_target = tf.convert_to_tensor(test_target)
# # test_target = (test_target+1)/2

In [83]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

Epoch 1/200


/Users/egemen/opt/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/2 [==============>...............] - ETA: 1s - loss: 0.7065 - accuracy: 0.5391
Epoch 1: val_accuracy improved from -inf to 0.62687, saving model to best_model.pkl


INFO:tensorflow:Assets written to: best_model.pkl/assets


INFO:tensorflow:Assets written to: best_model.pkl/assets


...

INFO:tensorflow:Assets written to: best_model.pkl/assets


2/2 [==============================] - 3s 3s/step - loss: 0.0912 - accuracy: 0.9851 - val_loss: 0.2202 - val_accuracy: 0.9478
Epoch 127/200
1/2 [==============>...............] - ETA: 0s - loss: 0.0939 - accuracy: 0.9844
Epoch 127: val_accuracy did not improve from 0.94776
2/2 [==============================] - 0s 68ms/step - loss: 0.0922 - accuracy: 0.9851 - val_loss: 0.2427 - val_accuracy: 0.9478
Epoch 128/200
1/2 [==============>...............] - ETA: 0s - loss: 0.1032 - accuracy: 0.9844
Epoch 128: val_accuracy did not improve from 0.94776
2/2 [==============================] - 0s 67ms/step - loss: 0.1025 - accuracy: 0.9851 - val_loss: 0.2619 - val_accuracy: 0.9254
Epoch 129/200
1/2 [==============>...............] - ETA: 0s - loss: 0.1139 - accuracy: 0.9531
Epoch 129: val_accuracy did not improve from 0.94776
2/2 [==============================] - 0s 68ms/step - loss: 0.1136 - accuracy: 0.9552 - val_loss: 0.2688 - val_accuracy: 0.9254
Epoch 130/200
1/2 [==============>............

INFO:tensorflow:Assets written to: best_model.pkl/assets


INFO:tensorflow:Assets written to: best_model.pkl/assets


2/2 [==============================] - 3s 3s/step - loss: 0.0750 - accuracy: 0.9851 - val_loss: 0.1717 - val_accuracy: 0.9552
Epoch 156/200
1/2 [==============>...............] - ETA: 0s - loss: 0.0935 - accuracy: 0.9844
Epoch 156: val_accuracy did not improve from 0.95522
2/2 [==============================] - 0s 67ms/step - loss: 0.0945 - accuracy: 0.9851 - val_loss: 0.1858 - val_accuracy: 0.9478
Epoch 157/200
1/2 [==============>...............] - ETA: 0s - loss: 0.1306 - accuracy: 0.9609
Epoch 157: val_accuracy did not improve from 0.95522
2/2 [==============================] - 0s 70ms/step - loss: 0.1264 - accuracy: 0.9627 - val_loss: 0.1777 - val_accuracy: 0.9403
Epoch 158/200
1/2 [==============>...............] - ETA: 0s - loss: 0.1103 - accuracy: 0.9766
Epoch 158: val_accuracy did not improve from 0.95522
2/2 [==============================] - 0s 67ms/step - loss: 0.1131 - accuracy: 0.9776 - val_loss: 0.1911 - val_accuracy: 0.9328
Epoch 159/200
1/2 [==============>............

In [84]:
#loading the model and checking accuracy on the test data
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict(test)
print(np.asarray(test_preds).reshape(-1)[8])
test_res = np.around(test_preds).reshape(-1)
print(np.asarray(test_res))
print(np.asarray(test_target))
# accuracy_score(test_target, test_preds)

3/3 [==============================] - 0s 10ms/step
0.9892462
[0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
[0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1.
 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]


In [320]:
import serial
import time
import csv

string=""
ser = serial.Serial('/dev/cu.usbmodem14301', 115200) #Tried with and without the last 3 parameters, and also at 1Mbps, same happens.
ser.flushInput()
ser.flushOutput()
data = []
num =len(data) 

temp = None
       
start_time = time.time()
seconds = 300
open('FanData', 'w').close()
while True:
    current_time = time.time()
    elapsed_time = current_time - start_time
    serialString = ser.readline()
    # Print the contents of the serial data
    temp=serialString.decode('Ascii')
    data.append(temp)
    with open('FanData', 'a') as f:
        f.write(str(current_time)+"\t")
        f.write(serialString.decode('Ascii'))
    print(serialString.decode('Ascii'))

    if elapsed_time > seconds:
        print("Finished iterating in: " + str(int(elapsed_time))  + " seconds")
        break

    

04

0,45,61,103

1,48,62,108

2,49,62,105

0,20,61,100

1,48,61,108

2,49,62,104

0,45,62,103

1,48,62,108

2,49,62,104

0,45,62,103

1,48,62,108

2,49,61,104

0,45,62,100

1,49,62,108

2,49,62,99

0,45,61,103

1,71,62,98

2,49,61,105

0,44,62,105

1,48,60,108

2,49,62,103

0,46,61,101

1,48,62,108

2,40,61,100

0,46,61,103

1,48,62,108

2,50,62,103

0,45,62,104

1,47,62,108

2,49,61,105

0,46,61,103

1,49,62,108

2,50,62,102

0,46,61,103

1,47,63,101

2,49,61,104

0,63,62,102

1,48,61,108

2,30,62,98

0,45,62,104

1,48,62,108

2,49,61,104

0,44,61,103

1,48,62,110

2,75,61,100

0,46,61,100

1,47,62,107

2,49,61,103

0,45,62,104

1,48,61,108

2,49,62,100

0,46,61,99

1,47,62,107

2,50,61,105

0,56,61,95

1,48,61,108

2,49,61,103

0,44,62,104

1,48,61,104

2,49,61,104

0,73,61,100

1,47,61,110

2,50,62,104

0,46,62,103

1,48,61,105

2,50,61,104

0,45,62,103

1,47,61,108

2,49,61,103

0,46,61,103

1,48,61,108

2,49,61,102

0,78,62,94

1,49,62,102

2,49,61,105

0,45,61,100

1,48,61,107

2

In [321]:
data = []
write = 0
sequence = []
with open('FanData') as datafile :
    for line in datafile.readlines():
        line = line.replace("\n", "")
        # if write:
        #     sequence.append(line.split("\t")[0])
        values = line.split("\t")[1].split(",")
        curr = []
        for i in values:
            if i.startswith(" Start Run: "):
                write = 1
            if i.startswith(" Finish Run: "):
                clas = i.replace(" Finish Run: ", "")
                write = 0
                if len(sequence) > 1:
                    data.append( [sequence, clas])
                    sequence = []
            if write and not i.startswith(" Start Run:"):
                curr.append(i)
        if len(curr) > 0:
            sequence.append(curr)

final_seq = np.stack(data)
# print(data)

/Users/egemen/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [322]:
data = []
targets = []
temp = []
for k in final_seq:
  temp = []
  for j in k[0]:
      temp.append(int(j[1]))
  targets.append(float(k[1]))
  temp = np.asarray(temp)
  temp = np.reshape(temp, (-1,3))
  data.append(temp[:45]) 
# print(temp[:45])
data = np.asarray(data)
data = np.reshape(data, (-1,1))
# print(data)

/Users/egemen/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [323]:
test = []
test_target = []
num =len(data) 
# print(len(data))
for i in range(num):
  # test_target.append(targets[i])
  test.append(data[i][0])

print(test)
test = tf.ragged.constant(test)
# test_target = tf.convert_to_tensor(test_target)

[array([[74, 49, 45],
       [48, 49, 46],
       [20, 79, 45],
       [47, 49, 46],
       [48, 50, 45],
       [48, 49, 44],
       [49, 49, 20],
       [48, 50, 46],
       [48, 50, 45],
       [47, 49, 46],
       [48, 50, 73],
       [49, 49, 44],
       [48, 49, 46],
       [48, 80, 46],
       [48, 19, 45],
       [47, 49, 45],
       [48, 61, 40],
       [27, 49, 38],
       [48, 50, 45],
       [99, 49, 44],
       [48, 49, 46],
       [43, 27, 45],
       [47, 49, 45],
       [48, 50, 45],
       [48, 49, 45],
       [48, 49, 76],
       [47, 50, 46],
       [48, 50, 46],
       [47, 49, 46],
       [84, 38, 45],
       [48, 49, 45],
       [49, 73, 46],
       [47, 63, 45],
       [48, 49, 46],
       [47, 49, 44],
       [13, 49, 44],
       [38, 49, 84],
       [48, 50, 45],
       [48, 49, 45],
       [48, 49, 45],
       [48, 50, 36],
       [40, 49, 44],
       [48, 50, 46],
       [48, 50, 45],
       [48, 49, 46]]), array([[48, 49, 45],
       [49, 50, 78],
       [48

In [324]:
model = load_model('best_model.pkl')

from sklearn.metrics import accuracy_score
test_preds = model.predict(test)
# print(np.asarray(test_preds).reshape(-1)[8])
test_res = np.around(test_preds).reshape(-1)
print(np.asarray(test_res))
print(np.asarray(test_target))

3/3 [==============================] - 0s 11ms/step
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0.]
[]
